In [ ]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 6.0 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116069 sha256=0b5759c8e21711e13d9d7787049cf2c9ff69cb1506397124977b59bc3b3a3a19
  Stored in directory: /root/.cache/pip/wheels/db/dd/76/acacd519878f133f2f869aec70db548d89e04013209c3c62bc
Successfully built neo4j


In [23]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import itertools
import os
import pandas as pd
from time import sleep
import re
from pandas.core.frame import DataFrame

In [24]:
class Gena:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
    
    def create_nodes(self, node_df):
        with self.driver.session() as session:
            for idx, row in node_df.iterrows():
                print(idx)
                unique_id = row['ID']
                name = row['Preferred label']
                label = row['Label'].upper()
                if ('No synonyms' not in row['Synonyms']) and (not isinstance(row['Synonyms'], float)):
                    synonyms = row['Synonyms']
                    create = "MERGE (n:" + label + " {id: $unique_id, name: $name, synonyms: $synonyms})"
                    session.run(create, unique_id=unique_id, name=name, synonyms=synonyms)
                else:
                    create = "MERGE (n:" + label + " {id: $unique_id, name: $name})"
                    session.run(create, unique_id=unique_id, name=name)
                
    def create_relationships(self, relation_df):
        with self.driver.session() as session:
            for idx, row in relation_df.iterrows():
                print(idx)
                if idx < 42616:
                    continue
                if idx%500 == 0 and idx != 0:
                    sleep(30)
                id_1 = row['ID_1']
                label_1 = row['Type_E1']
                id_2 = row['ID_2']
                label_2 = row['Type_E2']
                relation = re.sub(r'[^A-Za-z\s\d]', '', row['Relation'].upper()).replace(' ', '_')

                had_relation = [result for result in session.run("MATCH p = (node_1:" + label_1 + ")-[r:CID]->(node_2:" + label_2 + ") WHERE (node_1.id = $id_1 AND node_2.id = $id_2) RETURN nodes(p)", id_1=id_1, id_2=id_2)]
                if len(had_relation) > 0:
                    pmid_source = str(int(row['PMID']))
                    create_relation = "MATCH (node_1:" + label_1 + ")-[r:CID]->(node_2:" + label_2 + ")\
                    \nWHERE (node_1.id = $id_1 AND node_2.id = $id_2)\
                    \nSET r.detail = (CASE r.detail WHEN null THEN [$relation] ELSE r.detail + [$relation] END), r.source = r.source + [$pmid_source]"
                    session.run(create_relation, id_1=id_1, id_2=id_2, relation=relation, pmid_source=pmid_source)
                else:
                    if relation == "HAS_EXPOSURE_STRESSOR" or relation == "HAS_ALLERGIC_TRIGGER":
                        source = ["DOID"]
                    elif relation == "CID":
                        source = ["BC5CDR"]
                    else:
                        source = [str(int(row['PMID']))]
                    create_relation = f"MATCH (node_1:" + label_1 + "), (node_2:" + label_2 + f")\
                    \nWHERE (node_1.id = $id_1 AND node_2.id = $id_2)\
                    \nMERGE ((node_1)-[r:{relation}" + " {source:$source}" + "]->(node_2))"
                    session.run(create_relation, id_1=id_1, id_2=id_2, source=source)

    def delete_data(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

In [25]:
## Call GENA
uri = "neo4j+s://5cd8c3cd.databases.neo4j.io"
user = "neo4j"
password = "U1HpJgWOUHsEYhZkeixze6acMOvnOYAOfkv5PnuOt_U"
gena = Gena(uri, user, password)

In [26]:
# Create Relationship
relationships = pd.read_csv("/content/drive/MyDrive/gena_data_final.csv")
gena.create_relationships(relationships[::-1].reset_index(drop=True))
relationships.head(5)

Streaming output truncated to the last 5000 lines.
20033
20034
20035
20036
20037
20038
20039
20040
20041
20042
20043
20044
20045
20046
20047
20048
20049
20050
20051
20052
20053
20054
20055
20056
20057
20058
20059
20060
20061
20062
20063
20064
20065
20066
20067
20068
20069
20070
20071
20072
20073
20074
20075
20076
20077
20078
20079
20080
20081
20082
20083
20084
20085
20086
20087
20088
20089
20090
20091
20092
20093
20094
20095
20096
20097
20098
20099
20100
20101
20102
20103
20104
20105
20106
20107
20108
20109
20110
20111
20112
20113
20114
20115
20116
20117
20118
20119
20120
20121
20122
20123
20124
20125
20126
20127
20128
20129
20130
20131
20132
20133
20134
20135
20136
20137
20138
20139
20140
20141
20142
20143
20144
20145
20146
20147
20148
20149
20150
20151
20152
20153
20154
20155
20156
20157
20158
20159
20160
20161
20162
20163
20164
20165
20166
20167
20168
20169
20170
20171
20172
20173
20174
20175
20176
20177
20178
20179
20180
20181
20182
20183
20184
20185
20186
20187
20188
20189
20190
2

Failed to write data to connection ResolvedIPv4Address(('34.126.114.186', 7687)) (IPv4Address(('34.126.114.186', 7687)))


Streaming output truncated to the last 5000 lines.
39567
39568
39569
39570
39571
39572
39573
39574
39575
39576
39577
39578
39579
39580
39581
39582
39583
39584
39585
39586
39587
39588
39589
39590
39591
39592
39593
39594
39595
39596
39597
39598
39599
39600
39601
39602
39603
39604
39605
39606
39607
39608
39609
39610
39611
39612
39613
39614
39615
39616
39617
39618
39619
39620
39621
39622
39623
39624
39625
39626
39627
39628
39629
39630
39631
39632
39633
39634
39635
39636
39637
39638
39639
39640
39641
39642
39643
39644
39645
39646
39647
39648
39649
39650
39651
39652
39653
39654
39655
39656
39657
39658
39659
39660
39661
39662
39663
39664
39665
39666
39667
39668
39669
39670
39671
39672
39673
39674
39675
39676
39677
39678
39679
39680
39681
39682
39683
39684
39685
39686
39687
39688
39689
39690
39691
39692
39693
39694
39695
39696
39697
39698
39699
39700
39701
39702
39703
39704
39705
39706
39707
39708
39709
39710
39711
39712
39713
39714
39715
39716
39717
39718
39719
39720
39721
39722
39723
39724
3

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,NEW00000,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",NEW00001,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,NEW00000,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",NEW00001,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,NEW00002,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
relationships[::-1].reset_index(drop=True).iloc[2624:, :].head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
2624,23131487,We hypothesize that stimulation of β2-adrenerg...,D000068299,salmeterol,CHEMICAL,salmeterol,Salmeterol Xinafoate,"Xinafoate, Salmeterol|Salmeterol|Serevent",cause patient's,D015325,lactic acidosis,DISEASE,lactic acidosis,Pyruvate Dehydrogenase Complex Deficiency Disease,"PDH Deficiency|Deficiency, PDH|Ataxia, Intermi..."
2625,23131487,We report a sympathomimetic syndrome with meta...,D000068299,salmeterol,CHEMICAL,salmeterol,Salmeterol Xinafoate,"Xinafoate, Salmeterol|Salmeterol|Serevent",have intentional inhalation in,D013406,suicide attempt,MENTAL_HEALTH,suicide attempt,"Suicide, Attempted","Attempted Suicide|Suicide Attempt|Attempt, Sui..."
2626,34871985,The effect of herbal additives on avian coccid...,NEW22513,avian coccidiosis,DISEASE,avian coccidiosis,avian coccidiosis,No synonyms,facilitate repair of,NEW22514,epithelial injuries,DISEASE,epithelial injuries,epithelial injury,No synonyms
2627,34871985,The effect of herbal additives on avian coccid...,NEW22513,avian coccidiosis,DISEASE,avian coccidiosis,avian coccidiosis,No synonyms,be based due to presence of,http://purl.obolibrary.org/obo/CHEBI_32940,phenolic compounds,CHEMICAL,Alkylresorcinol,hexacosane,hexacosane|n-hexacosane|CH3-[CH2]24-CH3
2628,27529178,"Although strategic thinking on water, sanitati...",NEW22511,poultry feces-may,CHEMICAL,poultry feces-may,poultry feces-may,No synonyms,be important risk factor for,D053485,diarrhea,DISEASE,diarrhea,Porcine epidemic diarrhea virus,Coronavirus (1988-2006)
